# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [115]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from dotenv import find_dotenv

In [116]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [117]:
# Read the Mapbox API key
load_dotenv(find_dotenv())
map_box_api = os.getenv("MAPBOX_PUB_TOKEN")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [118]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
toronto_cens_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path, index_col="neighbourhood")

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [119]:
# Getting the data from the top 10 expensive neighbourhoods
toronto_top10_byhood = toronto_cens_data[
    ["neighbourhood","average_house_value"]].groupby(
    ["neighbourhood"]).mean().sort_values("average_house_value", ascending=False).head(10)

# Calculate the top 10 most expensive neighbourhoods per year
toronto_top10_peryr_byhood = toronto_cens_data[["neighbourhood","average_house_value","shelter_costs_owned"]
                 ].groupby(["year","neighbourhood"]
                          ).mean().sort_values(["year","average_house_value"], ascending=False).reset_index()
toronto_top10_2001_byhood = toronto_top10_peryr_byhood[toronto_top10_peryr_byhood["year"] == 2001].head(10)#.set_index("year")
toronto_top10_2006_byhood = toronto_top10_peryr_byhood[toronto_top10_peryr_byhood["year"] == 2006].head(10)#.set_index("year")
toronto_top10_2011_byhood = toronto_top10_peryr_byhood[toronto_top10_peryr_byhood["year"] == 2011].head(10)#.set_index("year")
toronto_top10_2016_byhood = toronto_top10_peryr_byhood[toronto_top10_peryr_byhood["year"] == 2016].head(10)#.set_index("year")
toronto_top10_peryr_byhood = pd.concat([toronto_top10_2001_byhood,toronto_top10_2006_byhood,
                                        toronto_top10_2011_byhood,toronto_top10_2016_byhood],axis='rows', join='inner').set_index("year")

# Calculate the mean number of dwelling types units per year
unit_types_per_yr = toronto_cens_data.drop(columns = ["average_house_value","shelter_costs_owned","shelter_costs_rented"]).groupby(['year']).sum()
unit_types_2001 = unit_types_per_yr.loc[2001]
unit_types_2006 = unit_types_per_yr.loc[2006]
unit_types_2011 = unit_types_per_yr.loc[2011]
unit_types_2016 = unit_types_per_yr.loc[2016]

# Calculate the average monthly shelter costs for owned and rented dwellings
shelter_costs_owned = toronto_cens_data[["shelter_costs_owned"]].groupby(['year']).mean()
shelter_costs_rented = toronto_cens_data[["shelter_costs_rented"]].groupby(['year']).mean()

# Calculate the mean values for each neighborhood
toronto_avg_value_byhood = toronto_cens_data.groupby(["neighbourhood"]).mean()
#df_neighbourhood_locations.set_index("neighbourhood", inplace=True)
toronto_value_location_byhood = pd.concat([df_neighbourhood_locations, toronto_avg_value_byhood], axis="columns", join="inner")

# Calculate Avg. House Val
average_house_value_df = toronto_cens_data[["average_house_value"]].groupby(['year']).mean()

# Calculate Avg. House Val by N'hood
average_house_val_byhood = toronto_cens_data[["average_house_value","neighbourhood"]].groupby(['year','neighbourhood']).mean()

# Calculate Dwelling Types by N'hood
average_dwllng_types_byhood = toronto_cens_data.reset_index().drop(
    columns=["average_house_value","shelter_costs_owned","shelter_costs_rented"]
    ).groupby(['year','neighbourhood']).mean()

# Calculate Avg. House Val by Year
avg_house_value_by_year = toronto_cens_data[["neighbourhood","average_house_value"]].reset_index()

### Panel Visualization Functions

In [127]:
# Define Panel visualization functions
def neighbourhood_map():
    toronto_val_map = px.scatter_mapbox(
        toronto_value_location_byhood,
        lat="lat",
        lon="lon",
        size="average_house_value",
        hover_data=["single_detached_house","apartment_five_storeys_plus","movable_dwelling",
                    "semi_detached_house","row_house","duplex","apartment_five_storeys_less",
                    "other_house","average_house_value","shelter_costs_owned","shelter_costs_rented"],
        color="average_house_value",
        title="Average House Values in Toronto",
        labels={'average_house_value':'Avg. House Value'},
        zoom=9.5,
        height=700,
        width=1000)
    return toronto_val_map

def create_bar_chart(data, title, xlabel, ylabel, color):
    # Plot the incoming data and return the plot
    bar_plot = data.hvplot.bar(
        xlabel=xlabel,
        ylabel=ylabel,
        title=title,
    # some standardised formatting options from here
        height=450,
        width=450,
        rot=90
    ).opts(yformatter="%.0f",
           color=color,
           line_color=color,
           fontscale=1.05)
    return bar_plot

def create_line_chart(data, title, xlabel, ylabel, color):
    # Plot the incoming data and return the plot
    line_plot = data.hvplot.line(
        xlabel=xlabel,
        ylabel=ylabel,
        title=title,
    # some standardised formatting options from here 
        width=550
    ).opts(
        yformatter="%.0f",
        color=color)
    return line_plot

# Not needed, as I used create_line_chart
#def average_house_value():
#    """Average house values per year."""
#    
#    # YOUR CODE HERE!

def average_value_by_neighbourhood(data, xlabel, ylabel, gby):
    avg_val_plot = data.hvplot(
        xlabel=xlabel,
        ylabel=ylabel,
        groupby=gby
        ).opts(yformatter="%0.0f")
    return avg_val_plot

def number_dwelling_types(data, xlabel, ylabel, gby):
    num_types_plot = data.hvplot.bar(
        stacked=False,
        xlabel=xlabel,
        ylabel=ylabel,
        y=['single_detached_house','apartment_five_storeys_plus',
             'movable_dwelling','semi_detached_house','row_house',
             'duplex','apartment_five_storeys_less','other_house'],
        groupby=gby,
        rot=90,
        height=500)
    return num_types_plot

def average_house_value_snapshot(data, xaxs, yaxs, colour, facet_r, title):
    avg_val_by_year = px.bar(
        data,
        x=xaxs,
        y=yaxs,
        color=colour,
        barmode="group",
        facet_row=facet_r,
        facet_row_spacing=0.025,
        title=title,
        labels={'average_house_value':'Avg. House Value','neighbourhood':''},
        height=950,
        width=1000)
    return avg_val_by_year

# Not needed as I created using create_bar_chart()
#def top_most_expensive_neighbourhoods():
#    """Top 10 most expensive neighbourhoods."""
#    
#    # YOUR CODE HERE!

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    
    # YOUR CODE HERE!


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [136]:
### Create a Title for the Dashboard
dash_title = "#Real Estate Analysis for Toronto: 2001-2016"

### Define a welcome text
tab_welcome_msg =   "Welcome to some real estate analysis for Toronto (Ontario, Canada)!  "  \
                    "Explore the visualisations on each tab, which show real estate analysis " \
                    "for a number of neighbourhoods, over a number of years. Feel free to zoom in" \
                    "and out, left and right, have a play and most importantly learn a little " \
                    "about the real estate market in Toronto!"

### Create a tab layout for the dashboard
welcome_column = pn.Column(tab_welcome_msg,neighbourhood_map())

### Yearly Market Analysis, creating each plot with colours
# then building the rows and column of rows
bar_2001_red = create_bar_chart(unit_types_2001,"Dwelling Types in Toronto - 2001","2001","Dwelling Type Units","red")
bar_2006_blue = create_bar_chart(unit_types_2006,"Dwelling Types in Toronto - 2006","2006","Dwelling Type Units","blue")
bar_2011_orange = create_bar_chart(unit_types_2011,"Dwelling Types in Toronto - 2011","2011","Dwelling Type Units","orange")
bar_2016_magenta = create_bar_chart(unit_types_2016,"Dwelling Types in Toronto - 2016","2016","Dwelling Type Units","magenta")
# Prep rows and column for the dashboard
yearly_mkt_analysis_r1 = pn.Row(bar_2001_red)
yearly_mkt_analysis_r1.append(bar_2006_blue)
yearly_mkt_analysis_r2 = pn.Row(bar_2011_orange)
yearly_mkt_analysis_r2.append(bar_2016_magenta)
yearly_mkt_analysis = pn.Column(yearly_mkt_analysis_r1,yearly_mkt_analysis_r2)

### Shelter Costs vs. House Value
# Creating the plots, then column for the dashboard
shelter_costs_owned_plt = create_line_chart(shelter_costs_owned,"Average Monthly Shelter Cost for Owned Dwellings in Toronto",
                                    "Year","Average Monthly Shelter Costs","blue")
shelter_costs_rented_plt = create_line_chart(shelter_costs_rented,"Average Monthly Shelter Cost for Rented Dwellings in Toronto",
                                       "Year","Average Monthly Shelter Costs","orange")
avg_house_value_plt = create_line_chart(average_house_value_df,"Average House Value in Toronto","Year","Average House Value","red")
# Prep column for the dashboard
shelter_costs = pn.Column(shelter_costs_owned_plt)
shelter_costs.append(shelter_costs_rented_plt)
shelter_costs.append(avg_house_value_plt)

### Neighbourhood Analysis
# Creating plots
hood_analysis_c1 = average_value_by_neighbourhood(average_house_val_byhood,"Year","Avg. House Value","neighbourhood")
hood_analysis_c2 = number_dwelling_types(average_dwllng_types_byhood,"Year","Dwelling Type Units","neighbourhood")
hood_analysis_r1 = average_house_value_snapshot(avg_house_value_by_year,"neighbourhood",
                             "average_house_value","average_house_value","year",
                             "Average House Values in Toronto per Neighbourhood")
hood_analysis_c = pn.Column(hood_analysis_c1)
hood_analysis_c.append(hood_analysis_c2)
hood_analysis = pn.Row(hood_analysis_c,hood_analysis_r1)

### Most Exy Neighbourhoods
toronto_most_exy_r1 = create_bar_chart(toronto_top10_byhood,"Top 10 Most Expensive Neighbourhoods in Toronto",
                                    "Neighbourhood","Avg. House Value","darkslateblue").opts(width=615,height=600)
toronto_most_exy = pn.Column(toronto_most_exy_r1)

### Create the main dashboard
###
dashboard = pn.Tabs(
    ("Welcome", welcome_column),
    ("Annual Market Analysis", yearly_mkt_analysis),
    ("Shelter Costs vs. House Value", shelter_costs),
    ("Neighbourhood Analysis", hood_analysis),
    ("Most Expensive Neighbourhoods", toronto_most_exy)
)
#dashboard


Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Column
        [0] Row
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
        [1] Row
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
    [2] Column
        [0] HoloViews(Curve)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [3] Row
        [0] Column
            [0] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                    [1] VSpacer()
            [1] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                    [1] VSpacer()
        [1] Plotly(Figure)
    [4] Column
        [0] HoloViews(Bars)

## Serve the Panel Dashboard

In [68]:
dashboard.servable(title=dash_title)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [69]:
# neighbourhood_map().show()

In [70]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [92]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [91]:
#avg_house_value

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()